# PhaLP

In [20]:
conn = DBInterface.connect(MySQL.Connection, "127.0.0.1", "root", "123", db = "PhaLP")

MySQL.Connection(host="127.0.0.1", user="root", port="3306", db="PhaLP")

## Look at the data

In [27]:
using DataFrames
tables = DataFrame(DBInterface.execute(conn, "SHOW TABLES;"))

Row,Tables_in_PhaLP
,String
1,CDSs
2,EC
3,UniParc
4,UniProt
5,domains
6,experimental_evidence
7,gene_ontologies
8,hosts
9,link_EC_UniProt
